In [0]:
from __future__ import annotations
from typing import Any, Dict, List

from dbx_utils.logging import getLogger

logger = getLogger(__name__)

# --------------------------------------------------------------------
# Widgets
# --------------------------------------------------------------------
dbutils.widgets.text("endpoint_table", "")        # e.g. "cat.schema.endpoint_table"
dbutils.widgets.text("task_value_key", "endpoints")  # key under which to store the list
dbutils.widgets.text("max_endpoints", "")         # optional: limit for testing (leave empty for all)

In [0]:
endpoint_table = dbutils.widgets.get("endpoint_table")
task_value_key = dbutils.widgets.get("task_value_key") or "endpoints"
max_endpoints_raw = dbutils.widgets.get("max_endpoints").strip()

if not endpoint_table:
    raise ValueError("Widget 'endpoint_table' is required.")

max_endpoints: int | None
if max_endpoints_raw:
    try:
        max_endpoints = int(max_endpoints_raw)
    except ValueError as exc:
        raise ValueError(f"Invalid 'max_endpoints': {max_endpoints_raw!r}") from exc
else:
    max_endpoints = None

logger.info("Preparing endpoint list from table: %s", endpoint_table)
logger.info("Task value key: %s", task_value_key)
logger.info("Max endpoints: %s", max_endpoints if max_endpoints is not None else "ALL")

In [0]:
# --------------------------------------------------------------------
# Read endpoint table
# --------------------------------------------------------------------
df = spark.read.table(endpoint_table).select("id", "endpoint", "params", "job_settings")

if max_endpoints is not None:
    df = df.limit(max_endpoints)

rows = df.collect()
logger.info("Fetched %d rows from endpoint table.", len(rows))

# --------------------------------------------------------------------
# Build list of endpoint payloads
# Each element will be used by a For Each loop in the job
# --------------------------------------------------------------------
EndpointPayload = Dict[str, Any]
payloads: List[EndpointPayload] = []

for row in rows:
    endpoint_id = row["id"]
    endpoint = row["endpoint"]
    params = row["params"] or {}
    job_settings = row["job_settings"] or {}

    payload: EndpointPayload = {
        "id": endpoint_id,
        "endpoint": endpoint,
        "params": params,
        "job_settings": job_settings,
    }
    payloads.append(payload)

logger.info("Built endpoint payload list with %d elements.", len(payloads))

# --------------------------------------------------------------------
# Store in task values for downstream For Each
# --------------------------------------------------------------------

dbutils.jobs.taskValues.set(
    key=task_value_key,
    value=payloads,
)

logger.info(
    "Stored endpoint payload list as task value with key '%s'.",
    task_value_key,
)

# Optional: return a small summary as notebook result
dbutils.notebook.exit(
    f"Prepared {len(payloads)} endpoints into task value '{task_value_key}'"
)